In [1]:
!pip install -r requirements.txt

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 43.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 44.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 46.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 38.2 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 25.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This

In [2]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import torch
torch.cuda.empty_cache()
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0))

CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 4090


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
check_point='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
tokenizer = AutoTokenizer.from_pretrained(check_point)
model = AutoModelForCausalLM.from_pretrained(check_point, torch_dtype=torch.float16,device_map="auto")
print(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151648: AddedToken("<think>", rstrip=False

In [4]:
!nvidia-smi

Sat May 10 04:04:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:81:00.0 Off |                  Off |
| 30%   33C    P2             56W /  450W |   15090MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [6]:
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<｜User｜>", "<｜Assistant｜>"]
})
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
tokenizer.pad_token = '<|pad|>'
print(tokenizer.pad_token_id)
print(tokenizer.eos_token_id)
model.resize_token_embeddings(len(tokenizer))

151665
151643


Embedding(151666, 3584)

In [7]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<|pad|>', 'additional_special_tokens': ['<｜User｜>', '<｜Assistant｜>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False

In [8]:
from datasets import load_dataset
raw_datasets=load_dataset('maxwell60701/genshin-impact-role-chat-model',download_mode='force_redownload')
raw_datasets

Using the latest cached version of the dataset since maxwell60701/genshin-impact-role-chat-model couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /output/huggingface/datasets/maxwell60701___genshin-impact-role-chat-model/default/0.0.0/2e804a4698e0dac30e5da8bf431249151072f447 (last modified on Fri May  9 09:26:48 2025).


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 4080
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 1020
    })
})

In [9]:
raw_datasets['train'][0]

{'messages': [{'content': '妮露什么性别', 'role': 'user'},
  {'content': '妮露是女性', 'role': 'assistant'}]}

In [10]:
raw_datasets['train'][0]['messages']

[{'content': '妮露什么性别', 'role': 'user'},
 {'content': '妮露是女性', 'role': 'assistant'}]

In [11]:
raw_train_datasets=raw_datasets['train']
raw_train_datasets

Dataset({
    features: ['messages'],
    num_rows: 4080
})

In [12]:
raw_test_datasets=raw_datasets['test']
raw_test_datasets

Dataset({
    features: ['messages'],
    num_rows: 1020
})

In [13]:
def tokenizer_convert(example):
  prompt=tokenizer.apply_chat_template(example['messages'],tokenize=False)
  return {'text':prompt}

In [14]:
train_datasets=raw_train_datasets.map(tokenizer_convert).remove_columns('messages')
train_datasets[0]['text']

'<｜begin▁of▁sentence｜><｜User｜>妮露什么性别<｜Assistant｜>妮露是女性<｜end▁of▁sentence｜>'

In [15]:
test_datasets=raw_test_datasets.map(tokenizer_convert).remove_columns('messages')
test_datasets[0]['text']

'<｜begin▁of▁sentence｜><｜User｜>班尼特用的是什么元素?<｜Assistant｜>火元素<｜end▁of▁sentence｜>'

In [16]:
import matplotlib.pyplot as plt
from transformers import TrainerCallback

In [17]:
class LossPlotCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []
        self.train_epoch = []
        self.eval_epoch = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        print(f"self: {self}", flush=True)
        print(f"logs: {logs}", flush=True)
        epoch = logs.get("epoch")
        if epoch is None:
            return
        if "loss" in logs:
            self.train_epoch.append(epoch)
            self.train_losses.append(logs["loss"])
        if "eval_loss" in logs:
            self.eval_epoch.append(epoch)
            self.eval_losses.append(logs["eval_loss"])
        self.plot_losses()

    def plot_losses(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.train_epoch, self.train_losses, label='Train Loss', marker='o')
        plt.plot(self.eval_epoch, self.eval_losses, label='Eval Loss', marker='x')
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.title("Training & Evaluation Loss")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig("loss_plot.png")
        plt.close()

In [18]:
from trl import SFTTrainer,SFTConfig

training_args=SFTConfig(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    bf16=True,
    logging_steps=10,
    output_dir="./",
    save_strategy="epoch",
    eval_strategy="epoch",
    num_train_epochs=10,
    learning_rate=5e-5,
    save_steps=100,
    push_to_hub=False,
    dataset_text_field="text",
    report_to="none",
)

In [19]:
from peft import LoraConfig,get_peft_model
peft_config=LoraConfig(
    r=32,  
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    target_modules=["embed_tokens", "lm_head","q_proj","k_proj","v_proj","o_proj"],
    task_type="CAUSAL_LM"
)

In [20]:
from trl import DataCollatorForCompletionOnlyLM
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer=tokenizer,
    instruction_template="<｜User｜>",
    response_template = "<｜Assistant｜>",
)
data_collator

DataCollatorForCompletionOnlyLM(tokenizer=LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', vocab_size=151643, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<|pad|>', 'additional_special_tokens': ['<｜User｜>', '<｜Assistant｜>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<｜User｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<｜Assistant｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|EOT|>", rstrip=False, lstrip=False, single_w

In [21]:
from transformers import EvalPrediction

def compute_metrics_fn(eval_pred: EvalPrediction) -> dict:
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    # 将预测结果转换为类别索引
    preds = predictions.argmax(axis=-1)
    # 计算准确率
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

trainer=SFTTrainer(
    model=model,
    train_dataset=train_datasets,
    eval_dataset=test_datasets,
    args=training_args,
    peft_config=peft_config,
    processing_class=tokenizer,
    data_collator=data_collator,
    callbacks=[LossPlotCallback()]
)

In [22]:
dataloader = trainer.get_train_dataloader()

for batch in dataloader:
    print(batch)
    break

{'input_ids': tensor([[151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665,
         151665, 151665, 151646, 151646, 151644, 100222,  99687, 102227, 102021,
         105319,     30, 151645, 100222,  99687, 102227,  20412,  47872,  99525,
          99235, 151643],
        [151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665,
         151665, 151665, 151665, 151665, 151646, 151646, 151644, 100222, 104654,
          99316, 104139, 102625, 104363,     30, 151645, 111548,   9370, 117216,
          99253, 151643],
        [151646, 151646, 151644, 108386,     11, 109194, 101176, 102071,  99079,
         104308, 102021, 105319,     30, 151645, 101176, 102071,  99079,  20412,
          60686,  99208, 107891,  99305,  88970,  99347, 107891,   3837,  57750,
         109753, 151643],
        [151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665, 151665,
         151665, 151665, 151665, 151646, 151646, 151644,  96465,  38212, 105198,
         104308, 

In [23]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
0,1.284100,0.956189
2,0.049800,0.071135
4,0.021000,0.019846
6,0.010600,0.016001
8,0.005300,0.018351
9,0.004000,0.018481


self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 9.5683, 'grad_norm': 397.1864013671875, 'learning_rate': 4.999235143132708e-05, 'num_tokens': 6212.0, 'mean_token_accuracy': 0.25335838296450675, 'epoch': 0.0784313725490196}
self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 5.0606, 'grad_norm': 36.019432067871094, 'learning_rate': 4.996941040535653e-05, 'num_tokens': 12419.0, 'mean_token_accuracy': 0.35233423225581645, 'epoch': 0.1568627450980392}
self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 3.6494, 'grad_norm': 21.881195068359375, 'learning_rate': 4.993119095936937e-05, 'num_tokens': 18731.0, 'mean_token_accuracy': 0.48620745316147806, 'epoch': 0.23529411764705882}
self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 2.8581, 'grad_norm': 14.633879661560059, 'learning_rate': 4.9877716479290174e-05, 'num_tokens': 24899.0, 'mean_token_accuracy': 0.5812658857554197, 'epoch': 0.313725490196078

/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 1.0362, 'grad_norm': 14.488682746887207, 'learning_rate': 4.871842768871928e-05, 'num_tokens': 81189.0, 'mean_token_accuracy': 0.8062988437712193, 'epoch': 1.0196078431372548}
self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 0.9013, 'grad_norm': 15.77553653717041, 'learning_rate': 4.85157282400961e-05, 'num_tokens': 87527.0, 'mean_token_accuracy': 0.8113073080778122, 'epoch': 1.0980392156862746}
self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 0.7643, 'grad_norm': 16.805570602416992, 'learning_rate': 4.829863985848587e-05, 'num_tokens': 93675.0, 'mean_token_accuracy': 0.8486723460257053, 'epoch': 1.1764705882352942}
self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'loss': 0.6902, 'grad_norm': 33.76056671142578, 'learning_rate': 4.806729537712017e-05, 'num_tokens': 99943.0, 'mean_token_accuracy': 0.8585738062858581, 'epoch': 1.2549019607843137}
sel

...

/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'eval_loss': 0.018480965867638588, 'eval_runtime': 8.8516, 'eval_samples_per_second': 115.233, 'eval_steps_per_second': 14.461, 'eval_num_tokens': 792699.0, 'eval_mean_token_accuracy': 0.9966032239608467, 'epoch': 9.96078431372549}


/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


self: <__main__.LossPlotCallback object at 0x7f9b55d3de70>
logs: {'train_runtime': 2175.637, 'train_samples_per_second': 18.753, 'train_steps_per_second': 0.584, 'total_flos': 4.317586829155123e+16, 'train_loss': 0.34050214814726176, 'epoch': 9.96078431372549}


TrainOutput(global_step=1270, training_loss=0.34050214814726176, metrics={'train_runtime': 2175.637, 'train_samples_per_second': 18.753, 'train_steps_per_second': 0.584, 'total_flos': 4.317586829155123e+16, 'train_loss': 0.34050214814726176})

In [24]:
trainer.save_model('genshin-impact-role-model')

/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [51]:
message='行秋什么身份，他性格特征是什么'

In [52]:
chat=[{"role":"user","content":message}]


In [27]:
from  peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#check_point='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
#tokenizer = AutoTokenizer.from_pretrained(check_point)
#model = AutoModelForCausalLM.from_pretrained(check_point, torch_dtype=torch.float16,device_map="cuda")

#tokenizer.add_special_tokens({
   # "additional_special_tokens": ["<｜User｜>", "<｜Assistant｜>"]
#})
#tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
#tokenizer.pad_token = '<|pad|>'
#print(tokenizer.pad_token_id)
#print(tokenizer.eos_token_id)
#model.resize_token_embeddings(len(tokenizer))
model=PeftModel.from_pretrained(model,'genshin-impact-role-model')
model.to('cuda')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(151666, 3584)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.05, inplace=False)
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 32x151666 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 3584x32 (cuda:0)])
          (lora_magnitude_vector): ModuleDict()
        )
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dro

In [53]:
prompt=tokenizer.apply_chat_template(chat,tokenize=True,add_generation_prompt=True,return_tensors='pt')
prompt=prompt.to(model.device)
print(prompt)

tensor([[151646, 151644,  22243, 100057,  99245, 101294,   3837,  42411, 102625,
         104363, 102021, 151645, 151648,    198]], device='cuda:0')


In [54]:
output=model.generate(prompt,
    temperature=0.1,
    top_p=0.1,
    top_k=10,
    max_length=512,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)

151643
151665


In [55]:
print(tokenizer.decode(output[0],skip_special_tokens=True))

行秋什么身份，他性格特征是什么<think>
行秋，行侠仗义的侠客
